In [6]:
import pandas as pd
import os
from glob import glob

# File paths
clinical_file = r"D:\mlpr data\Glioblastoma-ML-model\UPENN-GBM_clinical_info_v2.1.csv"
radiomics_folder = r"D:\mlpr data\radiomic_features_CaPTk"

# Load clinical data
clinical_df = pd.read_csv(clinical_file)
clinical_df.rename(columns={"ID": "PatientID"}, inplace=True)  # Standardizing ID column name

# Load all radiomic CSVs and merge horizontally on PatientID
radiomic_files = glob(os.path.join(radiomics_folder, "*.csv"))

# Initialize empty dataframe for radiomics
radiomics_df = pd.DataFrame()

for file in radiomic_files:
    df = pd.read_csv(file)
    df.rename(columns={"SubjectID": "PatientID"}, inplace=True)  # Standardizing ID column name
    
    # Merge radiomics files horizontally
    if radiomics_df.empty:
        radiomics_df = df
    else:
        radiomics_df = pd.merge(radiomics_df, df, on="PatientID", how="outer")

# Merge clinical data with radiomics data
merged_df = pd.merge(clinical_df, radiomics_df, on="PatientID", how="outer")

# Save final merged dataset
output_file = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
merged_df.to_csv(output_file, index=False)

print(f"Merged dataset saved at {output_file}")


Merged dataset saved at D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (forcing errors='coerce' turns non-numeric values into NaN)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED"])  # Drop ID and target
y = df["Survival_from_surgery_days_UPDATED"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoder for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

C:\Users\asp61\AppData\Local\Temp\ipykernel_26020\2633508008.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)  # Use all cores
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


Mean Absolute Error: 342.39
R² Score: 0.0764


In [4]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, explained_variance_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Root Mean Squared Error
evs = explained_variance_score(y_test, y_pred)  # Explained Variance Score

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Explained Variance Score: {evs:.4f}")


Mean Absolute Error: 342.39
R² Score: 0.0764
Root Mean Squared Error (RMSE): 475.31
Explained Variance Score: 0.0794


## Classifier

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (handling non-numeric values)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# **Convert Survival Days into Categories (Example Binning)**
# You can modify bins as per your requirement
bins = [0, 450, 1100, np.inf]  # <100 days, 100-365 days, >365 days

labels = [0, 1, 2]  # Class labels
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoders for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

C:\Users\asp61\AppData\Local\Temp\ipykernel_26020\1281962872.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.82      0.70        76
           1       0.36      0.25      0.29        40
           2       0.00      0.00      0.00        13

    accuracy                           0.56       129
   macro avg       0.32      0.36      0.33       129
weighted avg       0.47      0.56      0.50       129



c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric (handling non-numeric values)
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# **Convert Survival Days into Categories (Example Binning)**
bins = [0, 450, 1100, np.inf]  # Define bin edges
labels = [0, 1, 2]  # Class labels
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=bins, labels=labels)

# Separate features (X) and target (y)
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"]

# Identify categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convert to string before encoding
    label_encoders[col] = le  # Store encoders for future use

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce")  # Ensure all values are numeric
X = X.fillna(X.median())  # Replace NaNs with median

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.9)  # Retain 95% of variance
X_pca = pca.fit_transform(X_scaled)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42, stratify=y)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

# Print the number of PCA components
print(f"Number of PCA components used: {X_pca.shape[1]}")

C:\Users\asp61\AppData\Local\Temp\ipykernel_34328\3368345720.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.93      0.71        76
           1       0.17      0.03      0.04        40
           2       0.00      0.00      0.00        13

    accuracy                           0.56       129
   macro avg       0.25      0.32      0.25       129
weighted avg       0.39      0.56      0.43       129

Number of PCA components used: 182


c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

print(f"Original dataset shape: {df.shape}")

# Ensure target column exists
if "Survival_from_surgery_days_UPDATED" not in df.columns:
    print("Error: Target column 'Survival_from_surgery_days_UPDATED' not found!")
    print("Available columns:", df.columns)
    exit()

# Data cleaning - convert target column to numeric
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop rows where target variable is NaN
initial_count = df.shape[0]
df.dropna(subset=["Survival_from_surgery_days_UPDATED"], inplace=True)
print(f"Dropped {initial_count - df.shape[0]} rows with missing target values")

# === DATA TYPE INSPECTION AND FIXING ===
# First, let's check for mixed types in columns
def check_mixed_types(dataframe):
    mixed_cols = []
    for col in dataframe.columns:
        # Check if column has mixed types
        types = dataframe[col].apply(type).unique()
        if len(types) > 1:
            mixed_cols.append((col, types))
    return mixed_cols

# Identify mixed type columns
mixed_type_cols = check_mixed_types(df)
print("\nColumns with mixed data types:")
for col, types in mixed_type_cols:
    print(f"- {col}: {[t.__name__ for t in types]}")

# Convert all object columns to string type to prevent issues
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str)

# Make sure there are no numeric columns with string values
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    # Check if column has any string values
    try:
        pd.to_numeric(df[col])
    except:
        print(f"Converting mixed column {col} to string")
        df[col] = df[col].astype(str)

# Feature engineering
# 1. Add log transformation for skewed survival days
df["Log_Survival_Days"] = np.log1p(df["Survival_from_surgery_days_UPDATED"])

# 2. Binning target variable into 3 categories based on clinical thresholds
# For GBM patients: poor (<6 months), intermediate (6-12 months), good (>12 months)
survival_thresholds = [0, 180, 365, float('inf')]  # 6 months, 1 year
labels = [0, 1, 2]  # Poor, Medium, Good survival
df["Survival_Category"] = pd.cut(df["Survival_from_surgery_days_UPDATED"], bins=survival_thresholds, labels=labels)

# Check class distribution
print("\nClass distribution:")
class_dist = df["Survival_Category"].value_counts(normalize=True)
print(class_dist)

# Separate features and target
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category", "Log_Survival_Days"])
y = df["Survival_Category"]

# === FEATURE SELECTION BY CORRELATION ===
# For numerical features, drop highly correlated features (>0.95)
numerical_features = X.select_dtypes(include=['int64', 'float64'])
if not numerical_features.empty:
    correlation = numerical_features.corr().abs()
    upper_tri = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
    print(f"\nDropping {len(to_drop)} highly correlated numerical features")
    X = X.drop(columns=to_drop, errors='ignore')

# Identify categorical and numerical columns again after clean-up
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nAfter cleaning and feature selection:")
print(f"- Categorical features: {len(categorical_cols)}")
print(f"- Numerical features: {len(numerical_cols)}")

# Better preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Changed from KNNImputer for stability
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Apply preprocessing
print("\nApplying preprocessing transformations...")
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Processed training data shape: {X_train_processed.shape}")

# Handle class imbalance with SMOTE if needed
class_counts = np.bincount(y_train.astype(int))
if max(class_counts) / min(class_counts) > 1.5:  # If imbalance ratio > 1.5
    print("\nApplying SMOTE for class balancing...")
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)
    print("Original training class distribution:")
    print(pd.Series(y_train).value_counts())
    print("Resampled training class distribution:")
    print(pd.Series(y_train_resampled).value_counts())
else:
    X_train_resampled, y_train_resampled = X_train_processed, y_train

# === MODEL SELECTION AND TRAINING ===
# Try multiple classifiers and choose the best one
classifiers = {
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
}

print("\nTraining and evaluating multiple models...")
best_accuracy = 0
best_model_name = None
best_model = None

for name, model in classifiers.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_processed)
    accuracy = balanced_accuracy_score(y_test, y_pred)
    print(f"{name} Balanced Accuracy: {accuracy:.4f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = name
        best_model = model

print(f"\nBest model: {best_model_name} (Balanced Accuracy: {best_accuracy:.4f})")

# Fine-tune the best model
if best_model_name == 'Random Forest':
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
elif best_model_name == 'Gradient Boosting':
    param_dist = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    }
else:  # XGBoost
    param_dist = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5]
    }

print(f"\nFine-tuning {best_model_name}...")
search = RandomizedSearchCV(
    classifiers[best_model_name],
    param_distributions=param_dist,
    n_iter=10,  # Reduced for quicker results
    cv=StratifiedKFold(3),  # Using 3-fold CV for speed
    scoring='balanced_accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train_resampled, y_train_resampled)
print(f"Best parameters: {search.best_params_}")
best_model = search.best_estimator_

# Evaluate the final model
y_pred = best_model.predict(X_test_processed)
accuracy = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\nFinal Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Balanced Accuracy: {balanced_acc:.4f}")
print("Classification Report:\n", report)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Poor', 'Medium', 'Good'], 
            yticklabels=['Poor', 'Medium', 'Good'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig("confusion_matrix.png")
plt.close()

# Feature importance
if hasattr(best_model, 'feature_importances_'):
    # Get feature names after preprocessing
    feature_names = []
    categorical_feature_names = []
    
    for name, transformer, cols in preprocessor.transformers_:
        if name == 'cat':
            # For categorical features, we need to get the one-hot encoded feature names
            ohe = transformer.named_steps.get('onehot')
            if ohe:
                try:
                    transformed_names = ohe.get_feature_names_out(cols)
                    categorical_feature_names.extend(transformed_names)
                except:
                    # If can't get feature names, use placeholders
                    categorical_feature_names.extend([f'cat_{i}' for i in range(ohe.n_features_in_)])
        else:
            # For numerical features, use the original column names
            feature_names.extend(cols)
    
    # Combine all feature names
    all_feature_names = categorical_feature_names + feature_names
    
    # Get feature importances from the model
    importances = best_model.feature_importances_
    
    # Match feature names with importances
    # Note: This might not perfectly match if the preprocessing changed the number of features
    if len(all_feature_names) == len(importances):
        feature_importance = pd.DataFrame({
            'Feature': all_feature_names,
            'Importance': importances
        })
        feature_importance = feature_importance.sort_values('Importance', ascending=False)
        
        # Display top features
        print("\nTop 15 important features:")
        print(feature_importance.head(15))
        
        # Plot feature importance
        plt.figure(figsize=(10, 8))
        top_features = feature_importance.head(15)
        sns.barplot(x='Importance', y='Feature', data=top_features)
        plt.title(f'Top 15 Features by Importance - {best_model_name}')
        plt.tight_layout()
        plt.savefig("feature_importance.png")
    else:
        print("\nCouldn't match feature names with importances. Lengths differ.")
        print(f"Feature names: {len(all_feature_names)}, Importances: {len(importances)}")

# Save the model and results
import joblib
joblib.dump(preprocessor, 'glioblastoma_preprocessor.pkl')
joblib.dump(best_model, 'glioblastoma_best_model.pkl')

print("\nModel training and evaluation complete. Files saved.")
print("To use this model for predictions:")
print("1. Load the saved preprocessor and model")
print("2. Preprocess new data with the preprocessor")
print("3. Use the model to make predictions")

Original dataset shape: (671, 9516)
Dropped 27 rows with missing target values

Columns with mixed data types:
- Time_since_baseline_preop: ['int', 'str']

Class distribution:
Survival_Category
2    0.526398
0    0.267081
1    0.206522
Name: proportion, dtype: float64

Dropping 4060 highly correlated numerical features

After cleaning and feature selection:
- Categorical features: 8
- Numerical features: 5446

Training set size: (483, 5454)
Test set size: (161, 5454)

Applying preprocessing transformations...
Processed training data shape: (483, 5508)

Applying SMOTE for class balancing...
Original training class distribution:
Survival_Category
2    254
0    129
1    100
Name: count, dtype: int64
Resampled training class distribution:
Survival_Category
0    254
1    254
2    254
Name: count, dtype: int64

Training and evaluating multiple models...

Training Random Forest...


  File "c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\asp61\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1038, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        pass_fds, cwd, env,
                        ^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
                        gid, gids, uid, umask,
                        ^^^^^^^^^^^^^^^^^^^^^^
                        start_new_session, process_group)
                

Random Forest Balanced Accuracy: 0.3956

Training Gradient Boosting...


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA  # PCA for dimensionality reduction
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Handle class imbalance
import xgboost as xgb  # Better model

# Load data
file_path = r"D:\mlpr data\Glioblastoma-ML-model\stackAndModel\merged_data.csv"
df = pd.read_csv(file_path)

# Convert target column to numeric
df["Survival_from_surgery_days_UPDATED"] = pd.to_numeric(df["Survival_from_surgery_days_UPDATED"], errors="coerce")

# Drop NaN in target variable
df = df.dropna(subset=["Survival_from_surgery_days_UPDATED"])

# Convert survival days into 4 quantile-based classes
df["Survival_Category"], bins = pd.qcut(df["Survival_from_surgery_days_UPDATED"], q=4, labels=[0, 1, 2, 3], retbins=True)

# Separate features and target
X = df.drop(columns=["PatientID", "Survival_from_surgery_days_UPDATED", "Survival_Category"])
y = df["Survival_Category"].astype(int)

# Encode categorical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Fill missing numeric values with median
X = X.apply(pd.to_numeric, errors="coerce").fillna(X.median())

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95)  # Keep 95% variance
X_pca = pca.fit_transform(X_scaled)

print(f"Original features: {X.shape[1]}, Reduced features after PCA: {X_pca.shape[1]}")

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_pca, y)

# Split data using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

# Train an optimized XGBoost classifier
clf = xgb.XGBClassifier(
    n_estimators=200, learning_rate=0.05, max_depth=6, 
    subsample=0.8, colsample_bytree=0.8, random_state=42
)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Display bin ranges
print("Quantile Bins:", bins)

# # Cross-validation score
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# cv_scores = cross_val_score(clf, X_resampled, y_resampled, cv=cv, scoring="accuracy")
# print(f"Cross-Validation Accuracy: {np.mean(cv_scores):.4f}")


C:\Users\asp61\AppData\Local\Temp\ipykernel_9240\1190965226.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Original features: 9514, Reduced features after PCA: 268


KeyboardInterrupt: 